# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [177]:
import pandas as pd
import numpy as np
import math
import json
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

from scipy.stats import zscore

import sqlite3

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

### Explore and Clean Data

**portfolio.json**
* reward (int) - reward given for completing an offer
* channels (list of strings)
* difficulty (int) - minimum required spend to complete an offer
* duration (int) - time for offer to be open, in days
* offer_type (string) - type of offer ie BOGO, discount, informational
* id (string) - offer id

In [179]:
portfolio.head()

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7



**profile.json**
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* age (int) - age of the customer
* id (str) - customer id
* became_member_on (int) - date when customer created an app account
* income (float) - customer's income

In [181]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


**transcript.json**
* person (str) - customer id
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* value - (dict of strings) - either an offer id or transaction amount depending on the record
* time (int) - time in hours since start of test. The data begins at time t=0

In [183]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [184]:
# We have multiple IDs, im going to rename them so its easier to see which relates to offer and which to customer for future joins.
portfolio.rename(columns={'id': 'offer_id'}, inplace=True)
profile.rename(columns={'id': 'customer_id'}, inplace=True)
transcript.rename(columns={'person': 'customer_id'}, inplace=True)

In [185]:
# View them again
portfolio.head()

,reward,channels,difficulty,duration,offer_type,offer_id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [186]:
profile.head()

,gender,age,customer_id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [187]:
transcript.head()

,customer_id,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [188]:
transcript.tail()

,customer_id,event,value,time
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.59},714
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714
306531,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,{'amount': 3.53},714
306533,c02b10e8752c4d8e9b73f918558531f7,transaction,{'amount': 4.05},714


In [189]:
# Grouping and viewing unique event values
event_counts = transcript['event'].value_counts()
print(event_counts)

event
transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: count, dtype: int64


In [190]:
# Cleaning the value column - seperating it so we know the offer id to join if needed
# Extracting offer_id from both keys into a single column
transcript['offer_id'] = transcript['value'].apply(
    lambda x: x.get('offer id') if isinstance(x, dict) and 'offer id' in x else (
        x.get('offer_id') if isinstance(x, dict) and 'offer_id' in x else None
    )
)

# Create column with amount
transcript['amount'] = transcript['value'].apply(lambda x: x.get('amount') if isinstance(x, dict) else 0)

In [191]:
# Checking for nulls in my new fields, offer_id and amount
transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   customer_id  306534 non-null  object 
 1   event        306534 non-null  object 
 2   value        306534 non-null  object 
 3   time         306534 non-null  int64  
 4   offer_id     167581 non-null  object 
 5   amount       138953 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 14.0+ MB


In [192]:
# Viewing rows where both offer_id and amount are null - should be none
null_rows = transcript[(transcript['offer_id'].isnull()) & (transcript['amount'].isnull())]
print(null_rows)


Empty DataFrame
Columns: [customer_id, event, value, time, offer_id, amount]
Index: []


## Creating a variable to show if a customer has both seen and completed an offer to use in my model.

In [194]:
# Identifying Response (1 = responded, 0 = not responded)
transcript['viewed_flag'] = (transcript['event'] == 'offer viewed').astype(int)
transcript['completed_flag'] = (transcript['event'] == 'offer completed').astype(int)

In [195]:
# Grouping and viewing unique event values to QA
print(transcript['viewed_flag'].value_counts())
print(transcript['completed_flag'].value_counts())

viewed_flag
0    248809
1     57725
Name: count, dtype: int64
completed_flag
0    272955
1     33579
Name: count, dtype: int64


In [196]:
transcript.head()

,customer_id,event,value,time,offer_id,amount,viewed_flag,completed_flag
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,0,0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,0,0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,2906b810c7d4411798c6938adc9daaa5,NaN,0,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,fafdcd668e3743c1bb461111dcafc2a4,NaN,0,0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,0,0


In [197]:
#Taking viewed records
viewed=transcript[transcript['viewed_flag']>0]
#print(viewed)

#Taking completed records
completed=transcript[transcript['completed_flag']>0]
#print(completed)

# Merge together on customer_id and offer_id to get a list of offer that were viewed and completed
completed = completed.merge(viewed, on=['customer_id', 'offer_id'], how='inner')

# Creating deduplicated list of completed offers by customers
completed = completed[['customer_id', 'offer_id']].drop_duplicates()

# Adding completed_flag column with value 1
completed['completed_flag_final'] = 1

In [198]:
completed.head()

,customer_id,offer_id,completed_flag_final
0,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,2906b810c7d4411798c6938adc9daaa5,1
1,fe97aa22dd3e48c8b143116a8403dd52,fafdcd668e3743c1bb461111dcafc2a4,1
2,629fc02d56414d91bca360decdfa9288,9b98b8c7a33c4b65b9aebfe6a799e6d9,1
3,676506bad68e4161b9bbaffeb039626b,ae264e3637204a6fb9bb56bc8210ddfd,1
4,8f7dd3b2afe14c078eb4f6e6fe4ba97d,4d5c57ea9a6940dd891ad53e9dbe8da0,1


In [199]:
# Drop completed and viewed columns from transcript data
transcript.drop(columns=['completed_flag', 'viewed_flag'], inplace=True)

# Merge this on to the original data
transcript = transcript.merge(completed, on=['customer_id', 'offer_id'], how='left')

#Fill NAs
# Recommended way to fill NaNs without chained assignment warning
transcript['completed_flag_final'] = transcript['completed_flag_final'].fillna(0)

In [200]:
transcript.completed_flag_final.value_counts()

completed_flag_final
0.0    217569
1.0     88965
Name: count, dtype: int64

## Looking at deduping the data for ease

In [202]:
transcript_dedup = transcript

# Drop columns I dont need
transcript_dedup.drop(columns=['event', 'value', 'amount','time'], inplace=True)

#deduplicate based on all
transcript_dedup = transcript_dedup.drop_duplicates()

In [203]:
# Group by 'offer_id' and 'customer_id' and get max of 'flag' and 'customer_id'
transcript_dedup = transcript_dedup.groupby(['offer_id', 'customer_id']).agg({
    'completed_flag_final': 'max'
#    ,'customer_id': 'max'
}).reset_index()

In [204]:
# Checking for duplicates
duplicate_offer_customer = transcript_dedup.duplicated(subset=['offer_id', 'customer_id']).sum()
print(f"Number of duplicate (offer_id, customer_id) pairs: {duplicate_offer_customer}")

Number of duplicate (offer_id, customer_id) pairs: 0


In [205]:
transcript_dedup.head()

,offer_id,customer_id,completed_flag_final
0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0011e0d4e6b944f998e987f904e8c1e5,1.0
1,0b1e1539f2cc45b7b9fa7c272da2e1d7,003d66b6608740288d6cc97a6903f4f0,0.0
2,0b1e1539f2cc45b7b9fa7c272da2e1d7,00426fe3ffde4c6b9cb9ad6d077a13ea,0.0
3,0b1e1539f2cc45b7b9fa7c272da2e1d7,0056df74b63b4298809f0b375a304cf4,0.0
4,0b1e1539f2cc45b7b9fa7c272da2e1d7,0063def0f9c14bc4805322a488839b32,0.0


In [206]:
# Checking for nulls
transcript_dedup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63288 entries, 0 to 63287
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   offer_id              63288 non-null  object 
 1   customer_id           63288 non-null  object 
 2   completed_flag_final  63288 non-null  float64
dtypes: float64(1), object(2)
memory usage: 1.4+ MB


## Merging with the other datasets

In [208]:
# Merging Data to allow for more feature choice
# Keep in mind this data is sitting at an offer level - de-dup later to make more sence to me
df = transcript_dedup.merge(profile, on='customer_id', how='left')

df['offer_id'] = df['offer_id'].astype(str).str.strip()
portfolio['offer_id'] = portfolio['offer_id'].astype(str).str.strip()

df = df.merge(portfolio, on='offer_id', how='left')

In [209]:
df.head()

,offer_id,customer_id,completed_flag_final,gender,age,became_member_on,income,reward,channels,difficulty,duration,offer_type
0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0011e0d4e6b944f998e987f904e8c1e5,1.0,O,40,20180109,57000.0,5,"[web, email]",20,10,discount
1,0b1e1539f2cc45b7b9fa7c272da2e1d7,003d66b6608740288d6cc97a6903f4f0,0.0,F,26,20170621,73000.0,5,"[web, email]",20,10,discount
2,0b1e1539f2cc45b7b9fa7c272da2e1d7,00426fe3ffde4c6b9cb9ad6d077a13ea,0.0,F,19,20160809,65000.0,5,"[web, email]",20,10,discount
3,0b1e1539f2cc45b7b9fa7c272da2e1d7,0056df74b63b4298809f0b375a304cf4,0.0,M,54,20160821,91000.0,5,"[web, email]",20,10,discount
4,0b1e1539f2cc45b7b9fa7c272da2e1d7,0063def0f9c14bc4805322a488839b32,0.0,None,118,20180615,NaN,5,"[web, email]",20,10,discount


In [210]:
# QA - Check for dups again after merging
duplicate_offer_customer = transcript_dedup.duplicated(subset=['offer_id', 'customer_id']).sum()
print(f"Number of duplicate (offer_id, customer_id) pairs: {duplicate_offer_customer}")

Number of duplicate (offer_id, customer_id) pairs: 0


## Checking for Nulls
I can see nulls in:
* Gender
* Income

In [212]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63288 entries, 0 to 63287
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   offer_id              63288 non-null  object 
 1   customer_id           63288 non-null  object 
 2   completed_flag_final  63288 non-null  float64
 3   gender                55222 non-null  object 
 4   age                   63288 non-null  int64  
 5   became_member_on      63288 non-null  int64  
 6   income                55222 non-null  float64
 7   reward                63288 non-null  int64  
 8   channels              63288 non-null  object 
 9   difficulty            63288 non-null  int64  
 10  duration              63288 non-null  int64  
 11  offer_type            63288 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 5.8+ MB


### Handling Null values in gender

In [214]:
# Looking at existing categories
df.gender.value_counts()

gender
M    31645
F    22808
O      769
Name: count, dtype: int64

In [215]:
# Treating missing as its own category - simple, preserves all data, and works well with one-hot encoding.
# This approach lets the model learn patterns associated with missingness — sometimes missing data has signal (e.g., privacy concerns, unregistered users).
df['gender'] = df['gender'].fillna('Unknown')

### Handling missing values in income

In [217]:
# Checking if the midding income values are random or if they are informative
df['income_missing'] = df['income'].isna().astype(int)

# Compare conversion rate (flag) between missing and non-missing income
print(df.groupby('income_missing')['completed_flag_final'].mean())


income_missing
0    0.425537
1    0.119142
Name: completed_flag_final, dtype: float64


#### I can see from this that the missing income is informative — it's not missing at random, and likely correlates with the target.
As a result of this I will not just use median to impute the values and will instead use a combination of a missing indicator and imputing with the median.

This is:
1. Preserving the information in the observed income.
2. Allowinge the model to learn the predictive power of the missing income by using a flag.

In [219]:
# 1. missing indicator
df['income_missing'] = df['income'].isna().astype(int)

# 2. impute with median
df['income'] = df['income'].fillna(df['income'].median())

In [220]:
# Checking I have no nulls left
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63288 entries, 0 to 63287
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   offer_id              63288 non-null  object 
 1   customer_id           63288 non-null  object 
 2   completed_flag_final  63288 non-null  float64
 3   gender                63288 non-null  object 
 4   age                   63288 non-null  int64  
 5   became_member_on      63288 non-null  int64  
 6   income                63288 non-null  float64
 7   reward                63288 non-null  int64  
 8   channels              63288 non-null  object 
 9   difficulty            63288 non-null  int64  
 10  duration              63288 non-null  int64  
 11  offer_type            63288 non-null  object 
 12  income_missing        63288 non-null  int64  
dtypes: float64(2), int64(6), object(5)
memory usage: 6.3+ MB


## Feature Engineering

In [222]:
# Convert entries channels to comma-separated strings
df['channels'] = df['channels'].apply(lambda x: ','.join(sorted(x)) if isinstance(x, list) else x)

In [223]:
# Adding a feature for how many days the customer has been a member. 
df['member_days'] = (pd.to_datetime('2022-01-01') - pd.to_datetime(df['became_member_on'], format='%Y%m%d')).dt.days

In [224]:
# Adding a feature to band income levels in to low, medium and high.
df['income_band'] = pd.qcut(df['income'], q=3, labels=['Low', 'Medium', 'High'])

In [225]:
# Adding a feature to band ages - Adding a seperate category for values that were missing
# Creating a df for valid rows where income is not missing
non_missing = df['income_missing'] != 1

# Applying qcut to those rows
df.loc[non_missing, 'income_band'] = pd.qcut(df.loc[non_missing, 'income'], q=3, labels=['Low', 'Medium', 'High'])

# Filling the rest with a missing label
df['income_band'] = df['income_band'].astype('category').cat.add_categories(['Missing'])
df['income_band'] = df['income_band'].fillna('Missing')

## Encoding categorical features

In [227]:
# Select only numeric columns (int or float)
numerical_features = df[df.select_dtypes(include=['int64', 'float64']).columns]

# Select only categorical features columns (int or float)
categorical_features = df.select_dtypes(include=['object', 'category']).columns
categorical_features = df[categorical_features.drop(['offer_id', 'customer_id'], errors='ignore')]

In [228]:
# Initialize OneHotEncoder, drop='first' to avoid multicollinearity, handle_unknown='ignore' for safety
ohe = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')

# Fit and transform the categorical features
categorical_encoded_array = ohe.fit_transform(categorical_features)

# Get new column names from the encoder
encoded_cols = ohe.get_feature_names_out(categorical_features.columns)

# Convert the array back to a DataFrame with proper column names
categorical_encoded = pd.DataFrame(categorical_encoded_array, columns=encoded_cols, index=df.index)

# Combine with numeric columns (drop original categorical columns first)
numerical_features = df.drop(columns=['gender', 'channels', 'offer_type', 'income_band'])
df_encoded = pd.concat([numerical_features, categorical_encoded], axis=1)

## Remove outliers

In [230]:
# Select only numeric columns (int or float)
numeric_cols = df_encoded.select_dtypes(include=['int64', 'float64']).columns

# Calculate z-scores
#z_scores = df_encoded[numeric_cols].apply(zscore)

# Calculate z-scores manually - Due to Kernel crashing
z_scores = (df_encoded[numeric_cols] - df_encoded[numeric_cols].mean()) / df_encoded[numeric_cols].std()

# Filter rows
df_encoded = df_encoded[(np.abs(z_scores) < 3).all(axis=1)]

## Save new data to SQL database to import into next notebook

In [232]:
# Create SQLite database file
conn = sqlite3.connect("df_encoded.db")

# Save DataFrame to a table
df_encoded.to_sql("df_encoded", conn, if_exists="replace", index=False)

# Close connection
conn.close()